In [1]:
# default values of file paths, assuming quantum_fog is working dir
in_bif = 'examples_cbnets/earthquake.bif'
in_dot = 'examples_cbnets/earthquake.dot'
in_csv = 'learning/training_data_c/earthquake.csv'
qfog_path = None

**Parameter Learning Template**

# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Learned-Network-Parameters" data-toc-modified-id="Learned-Network-Parameters-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Learned Network Parameters</a></div><div class="lev2 toc-item"><a href="#QFog" data-toc-modified-id="QFog-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>QFog</a></div><div class="lev2 toc-item"><a href="#bnlearn" data-toc-modified-id="bnlearn-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>bnlearn</a></div><div class="lev2 toc-item"><a href="#Summary-of-Running-Times" data-toc-modified-id="Summary-of-Running-Times-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Summary of Running Times</a></div>

In [2]:
import pandas as pd
import numpy as np
from graphviz import Source

import warnings
warnings.filterwarnings("ignore", module="rpy2")

import rpy2
%load_ext rpy2.ipython
%R library("bnlearn");
%R library("Rgraphviz");

In [3]:
import os
import sys
if not qfog_path:
    os.chdir('../../')
else:
    os.chdir(qfog_path)
cwd = os.getcwd()
sys.path.insert(0,cwd)
print("cwd=", cwd)
from learning.NetParamsLner import *

cwd= /home/jupyter/Notebooks/Quantum/quantum-fog


Read in_csv, create Pandas Dataframe with it, push dataframe into R

In [4]:
states_df = pd.read_csv(in_csv)
states_df.head()

,Earthquake,Burglary,Alarm,JohnCalls,MaryCalls
0,1,1,1,1,1
1,1,1,1,1,1
2,1,1,1,1,1
3,1,1,1,1,1
4,1,1,1,1,1


In [5]:
states_df.tail()

,Earthquake,Burglary,Alarm,JohnCalls,MaryCalls
4995,1,1,1,1,1
4996,1,1,1,0,1
4997,1,1,1,1,1
4998,1,1,1,1,1
4999,1,1,1,1,1


In [6]:
%Rpush states_df
%R str(states_df)

'data.frame':	5000 obs. of  5 variables:
 $ Earthquake: int  1 1 1 1 1 1 1 1 1 1 ...
 $ Burglary  : int  1 1 1 1 1 1 1 1 1 1 ...
 $ Alarm     : int  1 1 1 1 1 1 1 1 1 1 ...
 $ JohnCalls : int  1 1 1 1 1 1 0 1 0 1 ...
 $ MaryCalls : int  1 1 1 1 1 1 1 1 1 1 ...


For bnlearn, data.frame columns cannot be int type, must be changed to factor.
data.frame is a list so can use lapply

In [7]:
%R states_df[] <- lapply(states_df, factor)
%R str(states_df)

'data.frame':	5000 obs. of  5 variables:
 $ Earthquake: Factor w/ 2 levels "0","1": 2 2 2 2 2 2 2 2 2 2 ...
 $ Burglary  : Factor w/ 2 levels "0","1": 2 2 2 2 2 2 2 2 2 2 ...
 $ Alarm     : Factor w/ 2 levels "0","1": 2 2 2 2 2 2 2 2 2 2 ...
 $ JohnCalls : Factor w/ 2 levels "0","1": 2 2 2 2 2 2 1 2 1 2 ...
 $ MaryCalls : Factor w/ 2 levels "0","1": 2 2 2 2 2 2 2 2 2 2 ...


# Learned Network Parameters

## QFog

In [8]:
# emp = empirical, learned
is_quantum = False
bnet = BayesNet.read_bif(in_bif, is_quantum)
bnet_emp = BayesNet.read_dot(in_dot)
vtx_to_states = bnet.get_vtx_to_state_names()
bnet_emp.import_nd_state_names(vtx_to_states)


lnr = NetParamsLner(is_quantum, bnet_emp, states_df)

In [9]:
%%capture qfog_params_time
%time lnr.learn_all_bnet_pots()

In [10]:
print(qfog_params_time)

****************un-normalizable pot
sick pot:
['Burglary', 'Earthquake', 'Alarm']
[[[  0.00000000e+00   0.00000000e+00]
  [  4.60000000e-03   7.40000000e-03]]

 [[  1.92000000e-02   1.60000000e-03]
  [  8.00000000e-04   9.66400000e-01]]]
mended pot:
 ['Burglary', 'Earthquake', 'Alarm']
[[[  0.00000000e+00   1.00000000e+00]
  [  4.60000000e-03   7.40000000e-03]]

 [[  1.92000000e-02   1.60000000e-03]
  [  8.00000000e-04   9.66400000e-01]]]
CPU times: user 30 ms, sys: 3.83 ms, total: 33.8 ms
Wall time: 38.5 ms



In [11]:
lnr.compare_true_and_emp_pots(bnet, bnet_emp)


node= Alarm
true:
['Burglary', 'Earthquake', 'Alarm']
[[[ 0.95   0.05 ]
  [ 0.29   0.71 ]]

 [[ 0.94   0.06 ]
  [ 0.001  0.999]]]
empirical:
['Burglary', 'Earthquake', 'Alarm']
[[[  0.00000000e+00   1.00000000e+00]
  [  3.83333333e-01   6.16666667e-01]]

 [[  9.23076923e-01   7.69230769e-02]
  [  8.27129859e-04   9.99172870e-01]]]

node= Earthquake
true:
['Earthquake']
[ 0.02  0.98]
empirical:
['Earthquake']
[ 0.0208  0.9792]

node= JohnCalls
true:
['Alarm', 'JohnCalls']
[[ 0.9   0.1 ]
 [ 0.05  0.95]]
empirical:
['Alarm', 'JohnCalls']
[[ 0.93495935  0.06504065]
 [ 0.05085093  0.94914907]]

node= MaryCalls
true:
['Alarm', 'MaryCalls']
[[ 0.7   0.3 ]
 [ 0.01  0.99]]
empirical:
['Alarm', 'MaryCalls']
[[ 0.68292683  0.31707317]
 [ 0.01086734  0.98913266]]

node= Burglary
true:
['Burglary']
[ 0.01  0.99]
empirical:
['Burglary']
[ 0.012  0.988]


## bnlearn


In [12]:
%Rpush in_bif
%R bn.fit = read.bif(in_bif)
%R str(bn.fit)

List of 5
 $ Burglary  :List of 4
  ..$ node    : chr "Burglary"
  ..$ parents : chr(0) 
  ..$ children: chr "Alarm"
  ..$ prob    : table [1:2(1d)] 0.01 0.99
  .. ..- attr(*, "dimnames")=List of 1
  .. .. ..$ : chr [1:2] "True" "False"
  ..- attr(*, "class")= chr "bn.fit.dnode"
 $ Earthquake:List of 4
  ..$ node    : chr "Earthquake"
  ..$ parents : chr(0) 
  ..$ children: chr "Alarm"
  ..$ prob    : table [1:2(1d)] 0.02 0.98
  .. ..- attr(*, "dimnames")=List of 1
  .. .. ..$ : chr [1:2] "True" "False"
  ..- attr(*, "class")= chr "bn.fit.dnode"
 $ Alarm     :List of 4
  ..$ node    : chr "Alarm"
  ..$ parents : chr [1:2] "Burglary" "Earthquake"
  ..$ children: chr [1:2] "JohnCalls" "MaryCalls"
  ..$ prob    : table [1:2, 1:2, 1:2] 0.95 0.05 0.29 0.71 0.94 0.06 0.001 0.999
  .. ..- attr(*, "dimnames")=List of 3
  .. .. ..$ Alarm     : chr [1:2] "True" "False"
  .. .. ..$ Burglary  : chr [1:2] "True" "False"
  .. .. ..$ Earthquake: chr [1:2] "True" "False"
  ..- attr(*, "class")= chr "b

In [13]:
%R bn = bn.net(bn.fit)
%R str(bn)

List of 3
 $ learning:List of 6
  ..$ whitelist: NULL
  ..$ blacklist: NULL
  ..$ test     : chr "none"
  ..$ ntests   : num 0
  ..$ algo     : chr "empty"
  ..$ args     : list()
 $ nodes   :List of 5
  ..$ Burglary  :List of 4
  .. ..$ mb      : chr [1:2] "Earthquake" "Alarm"
  .. ..$ nbr     : chr "Alarm"
  .. ..$ parents : chr(0) 
  .. ..$ children: chr "Alarm"
  ..$ Earthquake:List of 4
  .. ..$ mb      : chr [1:2] "Burglary" "Alarm"
  .. ..$ nbr     : chr "Alarm"
  .. ..$ parents : chr(0) 
  .. ..$ children: chr "Alarm"
  ..$ Alarm     :List of 4
  .. ..$ mb      : chr [1:4] "Burglary" "Earthquake" "JohnCalls" "MaryCalls"
  .. ..$ nbr     : chr [1:4] "Burglary" "Earthquake" "JohnCalls" "MaryCalls"
  .. ..$ parents : chr [1:2] "Burglary" "Earthquake"
  .. ..$ children: chr [1:2] "JohnCalls" "MaryCalls"
  ..$ JohnCalls :List of 4
  .. ..$ mb      : chr "Alarm"
  .. ..$ nbr     : chr "Alarm"
  .. ..$ parents : chr "Alarm"
  .. ..$ children: chr(0) 
  ..$ MaryCalls :List of 4
  .. ..

In [14]:
%%capture bnlearn_params_time
%time %R rfit = bn.fit(bn, data = states_df);

In [15]:
print(bnlearn_params_time)

CPU times: user 5.45 ms, sys: 3.78 ms, total: 9.23 ms
Wall time: 10.2 ms



In [16]:
%R str(rfit)

List of 5
 $ Burglary  :List of 4
  ..$ node    : chr "Burglary"
  ..$ parents : chr(0) 
  ..$ children: chr "Alarm"
  ..$ prob    : table [1:2(1d)] 0.012 0.988
  .. ..- attr(*, "dimnames")=List of 1
  .. .. ..$ : chr [1:2] "0" "1"
  ..- attr(*, "class")= chr "bn.fit.dnode"
 $ Earthquake:List of 4
  ..$ node    : chr "Earthquake"
  ..$ parents : chr(0) 
  ..$ children: chr "Alarm"
  ..$ prob    : table [1:2(1d)] 0.0208 0.9792
  .. ..- attr(*, "dimnames")=List of 1
  .. .. ..$ : chr [1:2] "0" "1"
  ..- attr(*, "class")= chr "bn.fit.dnode"
 $ Alarm     :List of 4
  ..$ node    : chr "Alarm"
  ..$ parents : chr [1:2] "Burglary" "Earthquake"
  ..$ children: chr [1:2] "JohnCalls" "MaryCalls"
  ..$ prob    : table [1:2, 1:2, 1:2] NaN NaN 0.9231 0.0769 0.3833 ...
  .. ..- attr(*, "dimnames")=List of 3
  .. .. ..$ Alarm     : chr [1:2] "0" "1"
  .. .. ..$ Burglary  : chr [1:2] "0" "1"
  .. .. ..$ Earthquake: chr [1:2] "0" "1"
  ..- attr(*, "class")= chr "bn.fit.dnode"
 $ JohnCalls :List of 4
 

In [17]:
%%R
j = 0
for( nd in nodes(bn.fit)){
    j = j + 1
    cat('----------------', nd[1], '\n')
    cat('true:\n')
    print(coef(bn.fit)[j])
    cat('empirical:\n')
    print(coef(rfit)[j])
}

---------------- Burglary 
true:
$Burglary
 True False 
 0.01  0.99 

empirical:
$Burglary
    0     1 
0.012 0.988 

---------------- Earthquake 
true:
$Earthquake
 True False 
 0.02  0.98 

empirical:
$Earthquake
     0      1 
0.0208 0.9792 

---------------- Alarm 
true:
$Alarm
, , Earthquake = True

       Burglary
Alarm    True False
  True  0.950 0.290
  False 0.050 0.710

, , Earthquake = False

       Burglary
Alarm    True False
  True  0.940 0.001
  False 0.060 0.999


empirical:
$Alarm
, , Earthquake = 0

     Burglary
Alarm 0            1
    0   0.9230769231
    1   0.0769230769

, , Earthquake = 1

     Burglary
Alarm            0            1
    0 0.3833333333 0.0008271299
    1 0.6166666667 0.9991728701


---------------- JohnCalls 
true:
$JohnCalls
         Alarm
JohnCalls True False
    True  0.90  0.05
    False 0.10  0.95

empirical:
$JohnCalls
         Alarm
JohnCalls          0          1
        0 0.93495935 0.05085093
        1 0.06504065 0.94914907

---------

## Summary of Running Times

In [18]:
print("QFog:\n", qfog_params_time)
print("bnlearn:\n", bnlearn_params_time)

QFog:
 ****************un-normalizable pot
sick pot:
['Burglary', 'Earthquake', 'Alarm']
[[[  0.00000000e+00   0.00000000e+00]
  [  4.60000000e-03   7.40000000e-03]]

 [[  1.92000000e-02   1.60000000e-03]
  [  8.00000000e-04   9.66400000e-01]]]
mended pot:
 ['Burglary', 'Earthquake', 'Alarm']
[[[  0.00000000e+00   1.00000000e+00]
  [  4.60000000e-03   7.40000000e-03]]

 [[  1.92000000e-02   1.60000000e-03]
  [  8.00000000e-04   9.66400000e-01]]]
CPU times: user 30 ms, sys: 3.83 ms, total: 33.8 ms
Wall time: 38.5 ms

bnlearn:
 CPU times: user 5.45 ms, sys: 3.78 ms, total: 9.23 ms
Wall time: 10.2 ms

